In [1]:
!pwd
import sys
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124\run_objective_4l


'c:\\users\\81908\\appdata\\local\\pypoetry\\cache\\virtualenvs\\tfgpu-ehdmne1y-py3.8\\scripts\\python.exe'

In [2]:
# tensorflow2.0 + kerasでGPUメモリの使用量を抑える方法(最小限だけ使うように設定)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
import sys
sys.path.append(r'C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124')
import moa_MLPs_funcs

Tensorflow version 2.3.1
n_classes: 206


In [4]:
import optuna


def objective(trial):
    model_type = "4l"
    params = {}
    params["activation"] = trial.suggest_categorical("activation", ["relu", "elu"])
    params["denses"] = trial.suggest_categorical(
        "denses",
        [
            [512, 448, 384, 320],
            [1024, 896, 768, 640],
            [2048, 1792, 1536, 1280],
            [2560, 2048, 1524, 1012],
            [512, 512, 512, 512],
            [1024, 1024, 1024, 1024],
            [1024, 768, 1536, 1792],
            [1024, 1536, 768, 1792],
            [512, 768, 1536, 1792],
            [512, 1536, 768, 512],
        ],
    )
    drop_rates = []
    for i in range(5):
        drop_rate = trial.suggest_uniform(f'drop_rate{i}', 0.2, 0.8)
        drop_rates.append(drop_rate)
    params["drop_rates"] = drop_rates
    print("-" * 100)
    print(f"params: {params}")

    oof_score, Y_pred = moa_MLPs_funcs.train_and_evaluate(model_type="4l", params=params)

    return oof_score

In [5]:
%%time

n_trials = 100
#n_trials = 50
#n_trials = 3

study = optuna.create_study(
    study_name="study",
    storage=f"sqlite:///study.db",
    load_if_exists=True,
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=1),
)
study.optimize(objective, n_trials=n_trials)
study.trials_dataframe().to_csv(f"objective_history.csv", index=False)
with open(f"objective_best_params.txt", mode="w") as f:
    f.write(str(study.best_params))
print(f"\nstudy.best_params:\n{study.best_params}")

[I 2020-11-24 15:14:14,297] A new study created in memory with name: study
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 448, 384, 320] which is of type list.
  warnings.warn(message)
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1024, 896, 768, 640] which is of type list.
  warnings.warn(message)
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but con

----------------------------------------------------------------------------------------------------
params: {'activation': 'elu', 'denses': [2560, 2048, 1524, 1012], 'drop_rates': [0.6321946960652949, 0.20006862489040694, 0.3813995435791039, 0.2880535344902678, 0.2554031568612787]}



c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Epoch 1/50
19/19 - 2s - loss: 0.2562 - binary_crossentropy: 0.2535 - val_loss: 7.7791 - val_binary_crossentropy: 7.7651
Epoch 2/50
19/19 - 1s - loss: 0.0303 - binary_crossentropy: 0.0228 - val_loss: 1.5225 - val_binary_crossentropy: 1.5115
Epoch 3/50
19/19 - 1s - loss: 0.0286 - binary_crossentropy: 0.0211 - val_loss: 0.2323 - val_binary_crossentropy: 0.2233
Epoch 4/50
19/19 - 1s - loss: 0.0266 - binary_crossentropy: 0.0200 - val_loss: 0.0545 - val_binary_crossentropy: 0.0469
Epoch 5/50
19/19 - 1s - loss: 0.0261 - binary_crossentropy: 0.0201 - val_loss: 0.0325 - val_binary_crossentropy: 0.0257
Epoch 6/50
19/19 - 1s - loss: 0.0259 - binary_crossentropy: 0.0201 - val_loss: 0.0285 - val_binary_crossentropy: 0.0222
Epoch 7/50
19/19 - 1s - loss: 0.0259 - binary_crossentropy: 0.0200 - val_loss: 0.0271 - val_binary_crossentropy: 0.0210
Epoch 8/50
19/19 - 1s - loss: 0.0260 - binary_crossentropy: 0.0202 - val_loss: 0.0264 - val_binary_crossentropy: 0.0203
Epoch 9/50
19/19 - 1s - loss: 0.0259 - b

Epoch 19/50
19/19 - 1s - loss: 0.0257 - binary_crossentropy: 0.0199 - val_loss: 0.0248 - val_binary_crossentropy: 0.0189
Epoch 20/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0198 - val_loss: 0.0248 - val_binary_crossentropy: 0.0189
Epoch 21/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0198 - val_loss: 0.0247 - val_binary_crossentropy: 0.0189
Epoch 22/50
19/19 - 1s - loss: 0.0257 - binary_crossentropy: 0.0199 - val_loss: 0.0247 - val_binary_crossentropy: 0.0189
Epoch 23/50
19/19 - 1s - loss: 0.0257 - binary_crossentropy: 0.0199 - val_loss: 0.0247 - val_binary_crossentropy: 0.0188
Epoch 24/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0198 - val_loss: 0.0247 - val_binary_crossentropy: 0.0188
Epoch 25/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0198 - val_loss: 0.0246 - val_binary_crossentropy: 0.0188
Epoch 26/50
19/19 - 1s - loss: 0.0256 - binary_crossentropy: 0.0198 - val_loss: 0.0246 - val_binary_crossentropy: 0.0187
Epoch 27/50
19/19 - 1s - loss: 0

...

[I 2020-11-24 15:25:17,443] Trial 1 finished with value: 0.01744356453303943 and parameters: {'activation': 'relu', 'denses': [1024, 1024, 1024, 1024], 'drop_rate0': 0.6018476220820881, 'drop_rate1': 0.7613234424836193, 'drop_rate2': 0.7077865500116103, 'drop_rate3': 0.3879641101593651, 'drop_rate4': 0.5147288957437228}. Best is trial 1 with value: 0.01744356453303943.


oof_score: 0.01744356453303943
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
params: {'activation': 'relu', 'denses': [2048, 1792, 1536, 1280], 'drop_rates': [0.33774632823789535, 0.5206483453682103, 0.7483772147475398, 0.47432288479219303, 0.4584191403108251]}


c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 448, 384, 320] which is of type list.
  warnings.warn(message)
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1024, 896, 768, 640] which is of type list.
  warnings.warn(message)
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2048, 1792, 1536, 1280] which is of type list.
  warnings.warn(messa

Epoch 1/50
19/19 - 2s - loss: 0.2539 - binary_crossentropy: 0.2511 - val_loss: 0.1374 - val_binary_crossentropy: 0.1244
Epoch 2/50
19/19 - 2s - loss: 0.0316 - binary_crossentropy: 0.0237 - val_loss: 0.0554 - val_binary_crossentropy: 0.0449
Epoch 3/50


KeyboardInterrupt: 